In [27]:
# 패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
import xgboost as xgb
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 15
import warnings 
warnings.filterwarnings(action='ignore')

In [8]:
EDA = pd.read_csv('edafinal.csv')

In [9]:
EDA.head()

,gid,geometry,2018,2019,2020,2021,cctv,112,sec_light,safe_bell,child_home,kindergtn,elemidhigh,safety_inds,chil_safehome,parking,xcoord,ycoord
0,마라231909,MULTIPOLYGON (((128.8523707124823 35.205903552...,0,0,0,0,0,10,2,0,0,0,0,0,0,0,128.852927,35.206348
1,마라154986,MULTIPOLYGON (((128.7688788763629 35.276233217...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128.769436,35.276678
2,마라213944,MULTIPOLYGON (((128.833116749215 35.2376741072...,0,0,0,0,0,9,0,0,0,0,0,0,0,0,128.833674,35.238119
3,마마105025,MULTIPOLYGON (((128.7155376522062 35.311944657...,0,0,0,0,3,0,0,1,0,0,0,0,0,0,128.716094,35.312390
4,마라202940,MULTIPOLYGON (((128.8209718821295 35.234200772...,0,0,0,0,0,48,2,0,0,0,0,1,0,0,128.821529,35.234646


In [10]:
EDA1 = EDA[EDA['cctv'] >= 1]

In [11]:
EDA1.head()

,gid,geometry,2018,2019,2020,2021,cctv,112,sec_light,safe_bell,child_home,kindergtn,elemidhigh,safety_inds,chil_safehome,parking,xcoord,ycoord
3,마마105025,MULTIPOLYGON (((128.7155376522062 35.311944657...,0,0,0,0,3,0,0,1,0,0,0,0,0,0,128.716094,35.312390
6,마라142954,MULTIPOLYGON (((128.7552421328997 35.247523262...,0,0,0,0,1,21,4,0,0,0,0,0,0,0,128.755799,35.247968
20,마라281955,MULTIPOLYGON (((128.9080022709618 35.246743775...,176,156,122,125,5,397,6,1,1,0,0,0,0,0,128.908559,35.247188
22,마라273935,MULTIPOLYGON (((128.8989018157547 35.228816966...,306,312,247,213,4,234,0,1,0,0,0,0,0,0,128.899459,35.229261
26,마라186890,MULTIPOLYGON (((128.8026719911344 35.189317442...,0,0,0,0,3,15,5,1,0,0,0,0,0,0,128.803228,35.189762


# XGBoost

In [23]:
params = {
    'objective' : 'reg:linear',
    'eval_metric' : 'mae',
    'eta' : 0.001,
    'max_depth' : 10,
    'subsample' : 0.6,
    'colsample_bytree' : 0.6,
    'alpha' : 0.001,
    'random_state' : 2022,
    'silent' : True
}


In [24]:
x_train = EDA1[['2018', '2019', '2020', '2021','sec_light','safe_bell','child_home','kindergtn','elemidhigh','safety_inds','chil_safehome','parking','cctv']]
y_train = EDA1[['112']]
x_test  = EDA[['2018', '2019', '2020', '2021','sec_light','safe_bell','child_home','kindergtn','elemidhigh','safety_inds','chil_safehome','parking','cctv']]
y_test = EDA['112']

In [25]:
dev_x, val_x, dev_y, val_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 2022)

In [32]:
tr_data = xgb.DMatrix(dev_x,dev_y)
va_data = xgb.DMatrix(val_x,val_y)

watchlist = [(tr_data,'train'), (va_data,'valid')]

model_xgb = xgb.train(
    params = params,
    dtrain = tr_data,
    num_boost_round = 2000,
    evals = watchlist,
    maximize = False,
    early_stopping_rounds = 100,
    verbose_eval = 100
)

[20:08:37] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[20:08:37] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-mae:93.92896	valid-mae:80.09158
[100]	train-mae:87.07574	valid-mae:74.00214
[200]	train-mae:81.02947	valid-mae:68.65248
[300]	train-mae:75.84068	valid-mae:64.52842
[400]	train-mae:71.29326	valid-mae:61.32825
[500]	train-mae:67.29669	valid-mae:58.90794
[600]	train-mae:63.70160	valid-mae:56.99725
[700]	train-mae:60.47956	valid-mae:55.36959
[800]	train-mae:57.78838	valid-mae:54.31876
[900]	train-mae:55.30117	valid-mae:53.46587
[1000]	train-mae:53.09333	valid-mae:52.97430
[1100]	train-mae:51.22609	valid-mae:52.66

In [33]:
dtest = xgb.DMatrix(x_test)

In [34]:
pred4 = model_xgb.predict(dtest,ntree_limit=model_xgb.best_ntree_limit)

In [35]:
print("MAE:",mean_absolute_error(y_test, pred4))

MAE: 34.038990777784214


In [36]:
EDA['pred']=pred4

In [37]:
EDA.head()

,gid,geometry,2018,2019,2020,2021,cctv,112,sec_light,safe_bell,child_home,kindergtn,elemidhigh,safety_inds,chil_safehome,parking,xcoord,ycoord,pred
0,마라231909,MULTIPOLYGON (((128.8523707124823 35.205903552...,0,0,0,0,0,10,2,0,0,0,0,0,0,0,128.852927,35.206348,21.079021
1,마라154986,MULTIPOLYGON (((128.7688788763629 35.276233217...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,128.769436,35.276678,28.364265
2,마라213944,MULTIPOLYGON (((128.833116749215 35.2376741072...,0,0,0,0,0,9,0,0,0,0,0,0,0,0,128.833674,35.238119,28.364265
3,마마105025,MULTIPOLYGON (((128.7155376522062 35.311944657...,0,0,0,0,3,0,0,1,0,0,0,0,0,0,128.716094,35.312390,41.543266
4,마라202940,MULTIPOLYGON (((128.8209718821295 35.234200772...,0,0,0,0,0,48,2,0,0,0,0,1,0,0,128.821529,35.234646,29.590132


In [38]:
EDA2 = EDA.sort_values(by=['pred'],ascending=False)

In [39]:
EDA2.head()

,gid,geometry,2018,2019,2020,2021,cctv,112,sec_light,safe_bell,child_home,kindergtn,elemidhigh,safety_inds,chil_safehome,parking,xcoord,ycoord,pred
488,마라258935,MULTIPOLYGON (((128.882422588875 35.2290063205...,93,88,98,104,2,4998,2,0,0,0,0,16,0,0,128.882980,35.229451,2244.541260
2172,마라243939,MULTIPOLYGON (((128.8660036604245 35.232799242...,82,78,86,86,5,1880,10,1,0,0,0,21,0,0,128.866561,35.233244,1330.865479
5695,마라247971,MULTIPOLYGON (((128.8708839231963 35.261595912...,72,64,51,51,0,549,0,0,0,0,0,18,0,0,128.871441,35.262040,991.067871
6550,마라243938,MULTIPOLYGON (((128.865988546614 35.2318977900...,7,6,6,6,1,1347,0,1,0,0,0,15,1,0,128.866545,35.232342,729.356262
5659,마라243943,MULTIPOLYGON (((128.8660641223773 35.236405051...,6,6,12,6,0,673,0,0,0,0,0,19,0,0,128.866621,35.236850,608.799561


In [58]:
EDA3 = EDA2[['gid','xcoord','ycoord','pred','geometry']]

In [59]:
EDA3

,gid,xcoord,ycoord,pred,geometry
488,마라258935,128.882980,35.229451,2244.541260,MULTIPOLYGON (((128.882422588875 35.2290063205...
2172,마라243939,128.866561,35.233244,1330.865479,MULTIPOLYGON (((128.8660036604245 35.232799242...
5695,마라247971,128.871441,35.262040,991.067871,MULTIPOLYGON (((128.8708839231963 35.261595912...
6550,마라243938,128.866545,35.232342,729.356262,MULTIPOLYGON (((128.865988546614 35.2318977900...
5659,마라243943,128.866621,35.236850,608.799561,MULTIPOLYGON (((128.8660641223773 35.236405051...
...,...,...,...,...,...
296,마라272974,128.898964,35.264430,10.786969,MULTIPOLYGON (((128.8984068103482 35.263985749...
31281,마마225040,128.848293,35.324512,10.786969,MULTIPOLYGON (((128.8477351543746 35.324067570...
4355,마라219958,128.840474,35.250666,10.786969,MULTIPOLYGON (((128.8399169220049 35.250221751...
1373,마마225093,128.849087,35.372289,9.592507,MULTIPOLYGON (((128.8485290975987 35.371843992...


In [147]:
EDA4 = EDA3[EDA3['pred']>250]

In [148]:
EDA4

,gid,xcoord,ycoord,pred,geometry
488,마라258935,128.882980,35.229451,2244.541260,MULTIPOLYGON (((128.882422588875 35.2290063205...
2172,마라243939,128.866561,35.233244,1330.865479,MULTIPOLYGON (((128.8660036604245 35.232799242...
5695,마라247971,128.871441,35.262040,991.067871,MULTIPOLYGON (((128.8708839231963 35.261595912...
6550,마라243938,128.866545,35.232342,729.356262,MULTIPOLYGON (((128.865988546614 35.2318977900...
5659,마라243943,128.866621,35.236850,608.799561,MULTIPOLYGON (((128.8660641223773 35.236405051...
...,...,...,...,...,...
362,마라259940,128.884155,35.233945,251.931946,MULTIPOLYGON (((128.8835977461563 35.233500996...
5135,마라283951,128.910695,35.243557,251.669189,MULTIPOLYGON (((128.9101375256739 35.243112447...
2138,마라276954,128.903050,35.246350,250.877197,MULTIPOLYGON (((128.9024924993472 35.245906147...
274,마라237977,128.860541,35.267573,250.281845,MULTIPOLYGON (((128.8599836251191 35.267128654...


In [54]:
EDA4.to_csv('EDAFINAL3.csv',index=False)

### MAE

#### XGBoost : 34.13

LGBM : 42.72 

Catboost : 52.21

RandomForest : 55.78


In [55]:
import json
import folium
from folium import plugins
import geopandas as gpd
import matplotlib.pyplot as plt

In [67]:
EDA4

,gid,xcoord,ycoord,pred,geometry
488,마라258935,128.882980,35.229451,2244.541260,MULTIPOLYGON (((128.882422588875 35.2290063205...
2172,마라243939,128.866561,35.233244,1330.865479,MULTIPOLYGON (((128.8660036604245 35.232799242...
5695,마라247971,128.871441,35.262040,991.067871,MULTIPOLYGON (((128.8708839231963 35.261595912...
6550,마라243938,128.866545,35.232342,729.356262,MULTIPOLYGON (((128.865988546614 35.2318977900...
5659,마라243943,128.866621,35.236850,608.799561,MULTIPOLYGON (((128.8660641223773 35.236405051...
...,...,...,...,...,...
362,마라259940,128.884155,35.233945,251.931946,MULTIPOLYGON (((128.8835977461563 35.233500996...
5135,마라283951,128.910695,35.243557,251.669189,MULTIPOLYGON (((128.9101375256739 35.243112447...
2138,마라276954,128.903050,35.246350,250.877197,MULTIPOLYGON (((128.9024924993472 35.245906147...
274,마라237977,128.860541,35.267573,250.281845,MULTIPOLYGON (((128.8599836251191 35.267128654...


In [173]:
EDA4

,gid,xcoord,ycoord,pred
488,마라258935,128.882980,35.229451,2244.541260
2172,마라243939,128.866561,35.233244,1330.865479
5695,마라247971,128.871441,35.262040,991.067871
6550,마라243938,128.866545,35.232342,729.356262
5659,마라243943,128.866621,35.236850,608.799561
...,...,...,...,...
362,마라259940,128.884155,35.233945,251.931946
5135,마라283951,128.910695,35.243557,251.669189
2138,마라276954,128.903050,35.246350,250.877197
274,마라237977,128.860541,35.267573,250.281845


In [154]:
EDA4 = EDA4.drop(['geometry'],axis=1)

In [180]:
EDA5 = EDA4.reset_index()

In [182]:
EDA5 = EDA5.drop(['index'],axis=1)

In [194]:
EDA5['순번'] = EDA5['pred'].rank(method='first',ascending = False)

In [195]:
EDA5

,gid,xcoord,ycoord,pred,순번
0,마라258935,128.882980,35.229451,2244.541260,1.0
1,마라243939,128.866561,35.233244,1330.865479,2.0
2,마라247971,128.871441,35.262040,991.067871,3.0
3,마라243938,128.866545,35.232342,729.356262,4.0
4,마라243943,128.866621,35.236850,608.799561,5.0
...,...,...,...,...,...
102,마라259940,128.884155,35.233945,251.931946,103.0
103,마라283951,128.910695,35.243557,251.669189,104.0
104,마라276954,128.903050,35.246350,250.877197,105.0
105,마라237977,128.860541,35.267573,250.281845,106.0


In [197]:
EDA5 = EDA5.drop(['pred'],axis=1)

In [200]:
EDA5['x좌표(경도)']=EDA5['xcoord']
EDA5['y좌표(위도)']=EDA5['ycoord']

In [201]:
EDA5 = EDA5.drop(['xcoord','ycoord'],axis=1)

In [202]:
EDA5

,gid,순번,x좌표(경도),y좌표(위도)
0,마라258935,1.0,128.882980,35.229451
1,마라243939,2.0,128.866561,35.233244
2,마라247971,3.0,128.871441,35.262040
3,마라243938,4.0,128.866545,35.232342
4,마라243943,5.0,128.866621,35.236850
...,...,...,...,...
102,마라259940,103.0,128.884155,35.233945
103,마라283951,104.0,128.910695,35.243557
104,마라276954,105.0,128.903050,35.246350
105,마라237977,106.0,128.860541,35.267573


In [203]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [204]:
EDA5

,gid,순번,x좌표(경도),y좌표(위도)
0,마라258935,1.0,128.882980,35.229451
1,마라243939,2.0,128.866561,35.233244
2,마라247971,3.0,128.871441,35.262040
3,마라243938,4.0,128.866545,35.232342
4,마라243943,5.0,128.866621,35.236850
5,마라184895,6.0,128.801104,35.194293
6,마라247972,7.0,128.871456,35.262942
7,마라246970,8.0,128.870327,35.261151
8,마라241926,9.0,128.864167,35.221550
9,마라183894,10.0,128.799991,35.193404
